In [1]:
pip install --upgrade pip==24.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install trdg

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
def fix_utils_file():
    import trdg
    import os
    
    # Locate the utils.py file in the trdg package
    utils_path = os.path.join(os.path.dirname(trdg.__file__), "utils.py")

    # Read and fix the file
    with open(utils_path, 'r') as file:
        content = file.read()

    if "image_font.getsize(text)[1]" in content:
        content = content.replace(
            "return image_font.getsize(text)[1]",
            "left, top, right, bottom = image_font.getbbox(text)\n    return bottom"
        )

        with open(utils_path, 'w') as file:
            file.write(content)
        print("Fixed utils.py successfully!")
    else:
        print("No need to fix utils.py")
fix_utils_file()

No need to fix utils.py


In [6]:
import os
import subprocess
import shutil
import random
import tempfile
from PIL import Image
import numpy as np

# Set a fixed global seed for reproducibility
GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# Folder paths
input_file = './sample_data/turkish_words.txt'
output_base_dir = './output/distortions/'
fonts_dir = './fonts/hw_fonts/'

# Create directories
textures_dir = './textures/'
validated_textures_dir = './validated_textures/'
os.makedirs(textures_dir, exist_ok=True)
os.makedirs(validated_textures_dir, exist_ok=True)
os.makedirs(output_base_dir, exist_ok=True)

# Define scenario names based on distortion type
DISTORTION_TYPES = {
    1: "undistorted",
    2: "blurry",
    3: "low_resolution",
    4: "noisy",
    5: "rotated"
}

# Total target images and batch size
total_target_images = 50
batch_size = 50

def get_distortion_type():
    """Get user input for distortion type"""
    print("Choose distortion type:")
    print("1 - Clean (no distortion)")
    print("2 - Blurry images")
    print("3 - Low resolution images")
    print("4 - Images with Gaussian noise")
    print("5 - Rotated images (skew)")

    while True:
        try:
            choice = int(input("Enter your choice (1-5): "))
            if 1 <= choice <= 5:
                return choice
            else:
                print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Please enter a valid number.")

def setup_directory(distortion_type):
    """Create output directory based on distortion type"""
    scenario_name = DISTORTION_TYPES[distortion_type]
    scenario_dir = os.path.join(output_base_dir, scenario_name)
    os.makedirs(scenario_dir, exist_ok=True)
    return scenario_dir

def prepare_texture_images():
    """
    Check textures directory, validate images, and create a new directory with
    properly processed images that avoid transparency issues.
    """
    os.makedirs(validated_textures_dir, exist_ok=True)

    # Check if original textures directory exists and has files
    if not os.path.exists(textures_dir):
        print(f"Textures directory {textures_dir} does not exist! Creating it now.")
        os.makedirs(textures_dir, exist_ok=True)
        print(f"Please add background images to {textures_dir} before continuing.")
        return validated_textures_dir, False

    texture_files = [f for f in os.listdir(textures_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not texture_files:
        print(f"No image files found in {textures_dir}")
        print(f"Please add background images to {textures_dir} before continuing.")
        return validated_textures_dir, False

    print(f"Found {len(texture_files)} potential texture files. Processing...")
    valid_count = 0

    # Process each image file
    for img_file in texture_files:
        src_path = os.path.join(textures_dir, img_file)
        dst_path = os.path.join(validated_textures_dir, os.path.splitext(img_file)[0] + '.jpg')

        try:
            with Image.open(src_path) as img:
                # Handle palette images with transparency
                if img.mode == 'P' and 'transparency' in img.info:
                    img = img.convert('RGBA')

                # Convert to RGB mode to remove transparency
                if img.mode == 'RGBA':
                    background = Image.new('RGB', img.size, (255, 255, 255))
                    background.paste(img, mask=img.split()[3])
                    img = background
                elif img.mode != 'RGB':
                    img = img.convert('RGB')

                # Create a standard size canvas
                standard_size = (800, 600)
                new_img = Image.new('RGB', standard_size, (255, 255, 255))

                # Calculate position to center the original image
                position = ((standard_size[0] - img.width) // 2,
                           (standard_size[1] - img.height) // 2)

                # Paste original image onto the standard canvas
                new_img.paste(img, position)

                # Save as JPG (no transparency)
                new_img.save(dst_path, 'JPEG', quality=95)
                valid_count += 1

        except Exception as e:
            print(f"Error processing image {img_file}: {e}")

    print(f"Processed {valid_count} out of {len(texture_files)} texture images")

    if valid_count == 0:
        print("No valid textures found. Will use plain white backgrounds instead.")
        return validated_textures_dir, False

    return validated_textures_dir, valid_count > 0

def add_gaussian_noise(image_path):
    """Add Gaussian noise to an image"""
    try:
        img = Image.open(image_path)
        img_array = np.array(img)

        # Add Gaussian noise
        mean = 0
        std = 50  # Standard deviation (noise level)
        noise = np.random.normal(mean, std, img_array.shape)
        noisy_img_array = img_array + noise
        noisy_img_array = np.clip(noisy_img_array, 0, 255).astype(np.uint8)

        # Convert back to image
        noisy_img = Image.fromarray(noisy_img_array)
        noisy_img.save(image_path)
        return True
    except Exception as e:
        print(f"Error adding noise to image {image_path}: {e}")
        return False

def generate_images(distortion_type):
    """Generate images using TRDG with specified distortion type"""
    scenario_dir = setup_directory(distortion_type)
    scenario_name = DISTORTION_TYPES[distortion_type]

    print(f"\nGenerating {scenario_name} images...")
    print(f"Using random seed: {GLOBAL_SEED}")

    # Always prepare texture backgrounds
    validated_textures_dir, has_texture_backgrounds = prepare_texture_images()
    if not has_texture_backgrounds:
        print("No texture backgrounds available. Please add images to the textures directory.")
        print("Falling back to plain white background.")
        background_option = ['--background', '1']  # Plain white
    else:
        background_option = ['--background', '3', '--image_dir', validated_textures_dir]

    # Track the total number of images generated
    total_generated = 0
    scenario_labels = {}

    # Create batches
    remaining = total_target_images
    batch_number = 1

    while remaining > 0:
        current_batch_size = min(batch_size, remaining)
        print(f"Processing batch {batch_number} ({current_batch_size} images)...")

        # Create a temporary directory for this batch
        with tempfile.TemporaryDirectory() as temp_dir:
            # Build the basic command with common parameters
            cmd = [
                'trdg',
                '--input_file', input_file,
                '--output_dir', temp_dir,
                '--count', str(current_batch_size),
                '--language', 'tr',
                '--extension', 'png',
                '--word_split',
                '--font_dir', fonts_dir,
                '--name_format', '2',  # Use numeric naming with labels.txt
                '--text_color', '#000000',  # Pure black text
                '--margins', '15,15,15,15',  # Balanced margins
                '--space_width', '1.0',  # Normal space width
                '--alignment', '0',  # Left alignment
            ]

            # Add background options
            cmd.extend(background_option)

            # Add distortion-specific parameters
            if distortion_type == 1:  # Clean/Undistorted
                cmd.extend(['--format', '64'])  # Fixed height

            elif distortion_type == 2:  # Blurry
                cmd.extend([
                    '--format', '64',  # Fixed height
                    '--blur', '2',     # Blur level
                ])

            elif distortion_type == 3:  # Low resolution
                cmd.extend([
                    '--format', '40',  # Lower height for low resolution
                ])

            elif distortion_type == 4:  # Noisy (we'll add noise after generation)
                cmd.extend(['--format', '64'])  # Fixed height

            elif distortion_type == 5:  # Rotated
                cmd.extend([
                    '--format', '64',  # Fixed height
                    '--skew_angle', '16',  # More pronounced skew angle
                    '--random_skew'       # Apply random skew
                ])

            # Run the command
            try:
                print(f"Executing command: {' '.join(cmd)}")
                result = subprocess.run(cmd, check=True, capture_output=True, text=True)
                print(f"Command executed successfully")
            except subprocess.CalledProcessError as e:
                print(f"Error executing command: {e}")
                print(f"Error output: {e.stderr}")
                continue

            # Check generated files
            generated_files = [f for f in os.listdir(temp_dir) if f.endswith('.png')]
            print(f"Generated {len(generated_files)} images in temporary directory")

            # Read the labels file
            labels_path = os.path.join(temp_dir, 'labels.txt')
            current_labels = {}

            if os.path.exists(labels_path):
                with open(labels_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        parts = line.strip().split(' ', 1)
                        if len(parts) == 2:
                            current_labels[parts[0]] = parts[1]

            # Copy, rename, and post-process files
            processed_count = 0
            for img_file, label in current_labels.items():
                # Create unique name
                scenario_img_num = total_generated + int(img_file.split('.')[0])
                scenario_name_png = f"{scenario_img_num}.png"
                src_path = os.path.join(temp_dir, img_file)

                # Copy to scenario directory
                scenario_dst_path = os.path.join(scenario_dir, scenario_name_png)
                try:
                    shutil.copy2(src_path, scenario_dst_path)

                    # Apply additional post-processing if needed
                    if distortion_type == 4:  # Add Gaussian noise
                        add_gaussian_noise(scenario_dst_path)

                    scenario_labels[scenario_name_png] = label
                    processed_count += 1

                except Exception as e:
                    print(f"Error processing file {img_file}: {e}")

            # Update counts
            total_generated += processed_count
            remaining -= processed_count
            batch_number += 1

            print(f"Processed {processed_count} images in this batch")

    # Write the scenario labels file
    scenario_labels_path = os.path.join(scenario_dir, 'labels.txt')
    with open(scenario_labels_path, 'w', encoding='utf-8') as f:
        for img_file, label in sorted(scenario_labels.items(), key=lambda x: int(x[0].split('.')[0])):
            f.write(f"{img_file} {label}\n")

    print(f"Total images generated: {total_generated}")
    print(f"Labels saved to {scenario_labels_path}")

    return total_generated, scenario_dir

# Main execution
try:
    print(f"Using global random seed: {GLOBAL_SEED}")

    # Get distortion type from user
    distortion_type = get_distortion_type()

    # Generate images with selected distortion
    actual_count, scenario_dir = generate_images(distortion_type)

    print(f"\nDataset generation complete. Total images: {actual_count}")
    print(f"Dataset available at: {scenario_dir}")

except Exception as e:
    print(f"Error generating dataset: {e}")
    import traceback
    print(traceback.format_exc())

Using global random seed: 42
Choose distortion type:
1 - Clean (no distortion)
2 - Blurry images
3 - Low resolution images
4 - Images with Gaussian noise
5 - Rotated images (skew)

Generating rotated images...
Using random seed: 42
No image files found in ./textures/
Please add background images to ./textures/ before continuing.
No texture backgrounds available. Please add images to the textures directory.
Falling back to plain white background.
Processing batch 1 (50 images)...
Executing command: trdg --input_file ./sample_data/turkish_words.txt --output_dir C:\Users\LEGION\AppData\Local\Temp\tmpedns8js_ --count 50 --language tr --extension png --word_split --font_dir ./fonts/hw_fonts/ --name_format 2 --text_color #000000 --margins 15,15,15,15 --space_width 1.0 --alignment 0 --background 1 --format 64 --skew_angle 16 --random_skew
Command executed successfully
Generated 50 images in temporary directory
Processed 50 images in this batch
Total images generated: 50
Labels saved to ./outp